<img align="left" src = https://project.lsst.org/sites/default/files/Rubin-O-Logo_0.png width=250, style="padding: 10px"> 
<b>Catalog access via TAP </b> <br>
Last verified to run on <b>2021-06-25</b> with LSST Science Pipelines release <b>w_2021_25</b> <br>
Contact authors: Leanne Guy <br>
Credit: Originally developed by Leanne Guy in the context of the Rubin DP0.1 <br>
Target audience: All DP0 delegates. <br>
Container Size: medium <br>
Questions welcome at <a href="https://community.lsst.org/c/support/dp0">community.lsst.org/c/support/dp0</a> <br>
Find DP0 documentation and resources at <a href="https://dp0-1.lsst.io">dp0-1.lsst.io</a> <br>

The Rubin Science Platform provides QUERY access to the DP0.1 catalogs via TAP from jupyter notebooks. TAP is a Virtual Observatory protocol for access to catalog data. In this tutorial, we will learn how to explore the DP0.1 archive via TAP and execute complex queries to retrieve data. Full TAP documentation can be found [here](https://www.ivoa.net/documents/TAP/).

This notebook demonstrates how to:<br>
1. Create an instance of the TAP Service<br>
2. Explore the DP0.1 schema and tables using the TAP service<br>
3. Prepare and execute ADQL queries on the DP0.1 tables, and retrieve data for analysis<br>
4. Visualize and interact with the retrieved data to do science <br>

In [ ]:
# Ensure compliance with Rubin coding standards
#%load_ext pycodestyle_magic
#%flake8_on

In [ ]:
# Ignore warnings
import warnings
warnings.filterwarnings('ignore')

### 0. Setup 

Import python packages

In [ ]:
import numpy as np
import re

import pandas
pandas.set_option('display.max_rows', 20)

import bokeh
from bokeh.io import output_file, output_notebook, show
from bokeh.layouts import gridplot
from bokeh.models import ColumnDataSource, Range1d, HoverTool, Selection
from bokeh.plotting import figure, output_file

import holoviews as hv
from holoviews import streams
from holoviews.operation.datashader import datashade, dynspread, rasterize
from holoviews.plotting.util import process_cmap
hv.extension('bokeh')
output_notebook()

In [ ]:
# This should match the verified version listed at the start of the notebook
! eups list -s lsst_distrib

### 1. Explore the DP0.1 catalogs 

#### 1.1 Create an instance of the Rubin TAP Service

Table Access Procotol (TAP) provides standardized access to catalog data for discovery, search, and retrieval. Full <a href="http://www.ivoa.net/documents/TAP">documentation for TAP</a> is provided by the International Virtual Observatory Alliance (IVOA).

The TAP service uses a query language similar to SQL (Structured Query Langage) called ADQL (Astronomical Data Query Language). The <a href="http://www.ivoa.net/documents/latest/ADQL.html">documentation for ADQL</a> includes more information about syntax and keywords.

**Hazard Warning:** Not all ADQL functionality are supported yet in the DP0 RSP. 

In [ ]:
# Utility functions for working with the TAP service. 
from rubin_jupyter_utils.lab.notebook import get_tap_service, retrieve_query  
service = get_tap_service()
assert service is not None

#### 1.2 DC2 catalogs in DP0.1

To find out what catalogs exist, we will query the Rubin TAP schema. There is one schema (table collection) for all DP0.1 catalogs called  `dp01_dc2_catalogs`. This query also returns a description on the DP0.1 schema. 

In [ ]:
# Query to find out what schemas (table collections) are in the Rubin TAP_SCHEMA. 
query = "SELECT * FROM tap_schema.schemas "
#query = "SELECT * FROM tap_schema.schemas WHERE tap_schema.schemas.schema_name like 'dp0%' "

# Returns an TAPResults instance.
results = service.search(query)

# The query returns a TAPResults object that can be easily converted to an astropy table and displayed in a notebook. 
# All the DP0 tables are in the "dp01_dc2_catalogs" schema	
results.to_table().show_in_notebook()

The TAP schema name for the DP0.1 catalogs is `dp01_dc2_catalogs`. Now let's explore tables in the DP0.1 schema, ordering by their index in the database. This is the order in which they will appear presented to the user in the RSP Portal. We see the five tables in the DP0.1 schema, the same five tables that are presented via the Portal GUI., together with a description of each. 

In [ ]:
# Find the DP0 schema name and store as a variable
schema_names = results['schema_name']
for name in schema_names: 
    if re.search('dp01', name):
        dp01_schema_name = name
        break
print("DP0.1 schema is " + dp01_schema_name)

In [ ]:
# Prepare the query to explore the tables in the DP0.1 schema
query = "SELECT * FROM tap_schema.tables WHERE tap_schema.tables.schema_name = '" + dp01_schema_name + "' order by table_index ASC"

# Execute the query
results = service.search(query)

# The query returns a TAPResults object that can be easily converted to an astropy table and displayed in a notebook. 
results.to_table().show_in_notebook()

<br>
Here are some definitions to help delegates understand the contents of the TAP schema. 

* `schema` - database terminology for the abstract design that represents the storage of data in a database. 
* `tap_schema` - the specific schema describing the TAP service. All TAP services must support a set of tables in a schema named TAP_SCHEMA that describe the tables and columns included in the service.
* `table` - a collection of related data held in a table format in a database, e.g the object(dp01_dc2_catalogs.object) or position (dp01_dc2_catalogs.position) tables 
* `table collection` - a collection of tables. e.g `dp01_dc2_catalogs`	
* `results` - the query result set. The TAP service returns data from a query as a `TAPResults` object. Find more about `TAPResults` [here](https://pyvo.readthedocs.io/en/latest/api/pyvo.dal.TAPResults.html).

### 3. Querying the Object catalog

The Object catalog (dp01_dc2_catalogs.object) contains sources detected in the coadded images (also called stacked or combined images). The Object catalog is likely to be the catalog that is of the most interest to DP0 delgates. 

The `object` catalog is described in the <a href="https://arxiv.org/abs/2101.04855">DESC's DC2 data release note</a>, and more information about the simulated data in the <a href="https://ui.adsabs.harvard.edu/abs/2021ApJS..253...31L/abstract">DESC's DC2 paper</a>. 

#### 3.1 Specifying the maximum number of records to return
For debugging and testing queries, it is often useful to only return a few records for expediency. This can be done in one of two ways, setting the `TOP` field in a query, or setting the `maxec` parameter in the TAP service query. The two methods are identical. 

In [ ]:
# Define the maximum records to return
maxRec = 5

# Build a query to find object with extendedness = 0 and sort the returned result set by decreasing magnitude in the r band. Only return the first 5 results
query = "SELECT TOP " + str(maxRec) + " objectId, ra, dec, extendedness, mag_r, magerr_r, good " \
        "FROM dp01_dc2_catalogs.object " \
        "WHERE extendedness = 0 " \
        "AND mag_r < 24 " \
        "ORDER by mag_r DESC"

#Execute the query
results = service.search(query)
assert len(results) == 5

In [ ]:
#Execute the same query using the maxrec parameter instead of the TOP 
query = "SELECT objectId, ra, dec, extendedness, mag_r, magerr_r, good " \
        "FROM dp01_dc2_catalogs.object " \
        "WHERE extendedness = 0 " \
        "AND mag_r < 24 " \
        "ORDER by mag_r DESC"
results1 = service.search(query, maxrec=maxRec)
assert len(results1) == 5
results1.to_table().show_in_notebook()

In [ ]:
# Assert that the contents of the two tables are identical
from pandas.util.testing import assert_frame_equal
assert_frame_equal(results.to_table().to_pandas(), results1.to_table().to_pandas())

#### 2.2 Cone search around a point with specified radius

In [ ]:
query = "SELECT  DISTANCE (POINT('ICRS', 61.863, -35.79), POINT('ICRS', ra, dec))) as dist, ra, dec, mag_g, mag_i "\
        " FROM dp01_dc2_catalogs.object " \
        " WHERE 1=CONTAINS(POINT('ICRS', ra, dec), CIRCLE('ICRS', 61.863, -35.79, 0.05555555555555555)) " \
        " AND mag_g <24 AND mag_i <24 ORDER by dist ASC"
print(query)

In [ ]:
# For a more detailed analysis of data returned from a query, we will usually want to put the data in a pandas dataframe
results_data = results.to_table().to_pandas()
results_data

### 3. XXX - Cone search joining results with the truth infomation 

#### 3.1 Query
The following illustrates a selection from the object table in a circular region of 0.1 degree radius centered on (RA, Dec) = (62.0, -37.0) degrees and join the object table with the truth table

In [ ]:
query = "SELECT obj.objectId, obj.ra, obj.dec, obj.mag_g, obj.mag_r, obj.mag_i, "\
        "obj.mag_g_cModel, obj.mag_r_cModel, obj.mag_i_cModel,"\
        "obj.psFlux_g, obj.psFlux_r, obj.psFlux_i, obj.cModelFlux_g, "\
        "obj.cModelFlux_r, obj.cModelFlux_i, obj.tract, obj.patch, "\
        "obj.extendedness, obj.good, obj.clean, "\
        "truth.mag_r as truth_mag_r, truth.match_objectId, "\
        "truth.flux_g, truth.flux_r, truth.flux_i, truth.truth_type, "\
        "truth.match_sep, truth.is_variable "\
        "FROM dp01_dc2_catalogs.object as obj "\
        "JOIN dp01_dc2_catalogs.truth_match as truth "\
        "ON truth.match_objectId = obj.objectId "\
        "WHERE CONTAINS(POINT('ICRS', obj.ra, obj.dec),CIRCLE('ICRS', 62.0, -37.0, 0.1))=1 "\
        "AND truth.match_objectid >= 0 "\
        "AND truth.is_good_match = 1"
print(query)

In [ ]:
results = service.search(query).to_table().to_pandas()
assert len(results) == 14424, "Incorrect number of results returned"

### 4. Visualize and analyse the dataset

Now we will do some interactive analysis with the data we have above. We will use bokeh and holoviews to create interactive plots so that we can explore the dataset. 

4.1 Color-Magnitude Diagram 
We will use bokeh to plot a color-magnitude (g vs. g-i) and a color-color (r-i vs. g-r) diagram making use of the cModel magnitudes. We will use the advanced linking features of bokeh to link these 2 plots


Enter the values seen in the example below. We will use the “cModel” magnitudes, plotting g vs. g-i, to make a color-magnitude diagram.